In [ ]:
import pandas as pd
import numpy as np
import math
from sklearn.covariance import graphical_lasso
from statsmodels.tsa.stattools import coint
from statsmodels.regression.rolling import RollingOLS

In [ ]:
start_year = 2015
end_year = 2021
alpha_glasso = 0.8
strategy_duration = 1
glasso_duration = 2
regr_window = 30
norm_window = 60
entry_level = 2
stop_loss = 5
take_profit = 0
initial_margin = 0.5
method = "BL"

In [ ]:
df_constituents = pd.read_csv("https://raw.githubusercontent.com/datasets/s-and-p-500-companies/master/data/constituents.csv")
df_prices = pd.read_pickle("s&p500_final")

In [ ]:
df_log_returns = np.log(df_prices).diff(1)

In [ ]:
df_partial_correlations_list = list()
df_pairs_list = list()
for i in range(start_year, end_year):
    start_glasso_year = i
    end_glasso_year = i + glasso_duration - 1
    start_strategy_year = end_glasso_year + 1
    end_strategy_year = start_strategy_year + strategy_duration - 1
    df_log_returns_reduced = df_log_returns[df_log_returns.index.year >= start_glasso_year]
    df_log_returns_reduced = df_log_returns_reduced[df_log_returns_reduced.index.year <= end_glasso_year]
    df_log_returns_reduced = df_log_returns_reduced.dropna(axis = 1)
    df_log_returns_reduced = (df_log_returns_reduced - df_log_returns_reduced.mean()) / df_log_returns_reduced.std()
    cov_matrix = df_log_returns_reduced.cov()
    precision_matrix = pd.DataFrame(graphical_lasso(cov_matrix.to_numpy(), max_iter = 100, alpha = alpha_glasso)[1], columns = df_log_returns_reduced.columns)
    precision_matrix.index = df_log_returns_reduced.columns
    partial_correlations_matrix = precision_matrix.copy()
    pairs_list = list()
    for i in range(0, partial_correlations_matrix.shape[0]):
        for j in range(0, partial_correlations_matrix.shape[1]):
            partial_correlations_matrix.iloc[i][j] = - precision_matrix.iloc[i][j] / (math.sqrt(precision_matrix.iloc[i][i] * precision_matrix.iloc[j][j]))
            if partial_correlations_matrix.iloc[i][j] != 0 and i != j:
                pair_row = sorted([partial_correlations_matrix.index[i], partial_correlations_matrix.columns[j]])
                pair_row.append(partial_correlations_matrix.iloc[i][j])
                pairs_list.append(pair_row)
    np.fill_diagonal(partial_correlations_matrix.values, 0)
    df_pairs = pd.DataFrame(pairs_list, columns = ["Stock_X", "Stock_Y", "Correlation"])
    df_pairs[f"{start_glasso_year}_{end_glasso_year}"] = 1
    df_pairs.drop_duplicates(subset = ["Stock_X", "Stock_Y"], inplace = True)
    df_pairs_list.append(df_pairs)
    df_partial_correlations_list.append(partial_correlations_matrix)

In [ ]:
[df_pair.drop("Correlation", axis = 1, inplace = True) for df_pair in df_pairs_list]
[df_pair.set_index(["Stock_X", "Stock_Y"], inplace = True) for df_pair in df_pairs_list]
df_rolling_pairs = pd.DataFrame().join(other = df_pairs_list, how = "outer").fillna(0)

In [ ]:
df_rolling_pairs.to_pickle("Rolling pairs")

In [ ]:
df_rolling_pairs = pd.read_pickle("Rolling pairs")

In [ ]:
def level_crosses(series, level = 2):
    change = []
    for i, el in enumerate(series):
        if i != 0 and el > level and series[i-1] < level:
            change.append(1)
        elif i != 0 and el < level and series[i-1] > level:
            change.append(-1)
        else:
            change.append(0)
    return change

def reduce_df(df, start_year, end_year):
    df = df[df.index.year >= start_year]
    df = df[df.index.year <= end_year]
    return df

In [ ]:
df_prices = pd.read_pickle("s&p500_final")
df_bid_ask = pd.read_csv("stocks_bid_ask.csv")

In [ ]:
df_bid_ask["date"] = pd.to_datetime(df_bid_ask.date)

In [ ]:
df_pairs_returns_list = list()
for i in range(start_year, end_year):
    start_glasso_year = i
    end_glasso_year = i + glasso_duration - 1
    start_strategy_year = end_glasso_year + 1
    end_strategy_year = start_strategy_year + strategy_duration - 1
    print(start_glasso_year, end_glasso_year, start_strategy_year, end_strategy_year)
    df_prices_glasso = reduce_df(df_prices, start_glasso_year, end_glasso_year)
    df_prices_strategy = reduce_df(df_prices, start_strategy_year, end_strategy_year)
    glasso_pairs = df_rolling_pairs[df_rolling_pairs[f"{start_glasso_year}_{end_glasso_year}"] == 1].index
    df_pairs_returns = pd.DataFrame()
    for pair in glasso_pairs:
        engle_granger = coint(
            df_prices_glasso[pair[0]], 
            df_prices_glasso[pair[1]]
        )
        p_value = engle_granger[1]
        if p_value < 0.05:
            rols = RollingOLS(df_prices[pair[1]], df_prices[pair[0]], window = regr_window)
            rres = rols.fit()
            beta = rres.params
            df_pair = pd.DataFrame()
            df_pair["Beta"] = rres.params
            df_pair["Spread"] = df_prices[pair[1]] - df_pair["Beta"] * df_prices[pair[0]]
            df_pair["Normalized_Spread"] = (df_pair["Spread"] - df_pair["Spread"].rolling(norm_window).mean()) / df_pair["Spread"].rolling(norm_window).std()
            df_pair_reduced = reduce_df(df_pair, start_strategy_year, end_strategy_year)
            upper_trading = level_crosses(df_pair_reduced["Normalized_Spread"], level = entry_level)
            lower_trading = level_crosses(df_pair_reduced["Normalized_Spread"], level = -entry_level)
            mean = level_crosses(df_pair_reduced["Normalized_Spread"], level = take_profit)
            upper_stop_loss = level_crosses(df_pair_reduced["Normalized_Spread"], level = stop_loss)
            lower_stop_loss = level_crosses(df_pair_reduced["Normalized_Spread"], level = -stop_loss)
            long_X = 0
            returns = list()
            df_bid_ask_X = df_bid_ask[df_bid_ask["TICKER"] == pair[0]].set_index("date")
            df_bid_ask_Y = df_bid_ask[df_bid_ask["TICKER"] == pair[1]].set_index("date")
            stock_X = reduce_df(df_bid_ask_X, start_strategy_year, end_strategy_year)
            stock_Y = reduce_df(df_bid_ask_Y, start_strategy_year, end_strategy_year)
            trade_allowed = True
            if len(stock_X) == len(stock_Y) == len(df_pair_reduced):
                for i, (signal_upper, signal_lower, signal_mean, stop_loss_upper, stop_loss_lower) in enumerate(zip(upper_trading, lower_trading, mean, upper_stop_loss, lower_stop_loss)):
                    if (not trade_allowed) and abs(signal_mean) == 1:
                        trade_allowed = True
                    if long_X != 0:
                        stock_X_price = stock_X["BID"].iloc[i] if long_X == 1 else stock_X["ASK"].iloc[i]
                        stock_Y_price = stock_Y["ASK"].iloc[i] if long_X == 1 else stock_Y["BID"].iloc[i]
                        stock_X_previous_price = stock_X["BID"].iloc[i-1] if long_X == 1 else stock_X["ASK"].iloc[i-1]
                        stock_Y_previous_price = stock_Y["ASK"].iloc[i-1] if long_X == 1 else stock_Y["BID"].iloc[i-1]
                        profit_X = beta * long_X * (stock_X_price - stock_X_previous_price)
                        profit_Y = long_X * (stock_Y_previous_price - stock_Y_price)
                        if method == "MO":
                            trade_cost = initial_margin * stock_Y_entry_price if long_X == 1 else beta * initial_margin * stock_X_entry_price
                        elif method == "BL":
                            trade_cost = beta * stock_X_entry_price + initial_margin * stock_Y_entry_price if long_X == 1 else stock_Y_entry_price + beta * initial_margin * stock_X_entry_price
                        daily_return = (profit_X + profit_Y) / trade_cost
                        returns.append(daily_return)
                        if abs(signal_mean) == 1 or stop_loss_upper == 1 or stop_loss_lower == -1:
                            long_X = 0
                            if stop_loss_upper == 1 or stop_loss_lower == -1:
                                trade_allowed = False
                    else:
                        returns.append(math.nan)
                        if (signal_upper == -1 or signal_lower == 1) and trade_allowed:
                            long_X = 1 if signal_upper == -1 else -1
                            stock_X_entry_price = stock_X["ASK"].iloc[i] if long_X == 1 else stock_X["BID"].iloc[i]
                            stock_Y_entry_price = stock_Y["BID"].iloc[i] if long_X == 1 else stock_Y["ASK"].iloc[i]
                            beta = df_pair_reduced["Beta"].iloc[i]
                df_pairs_returns[f"{pair[0]}_{pair[1]}"] = pd.Series(returns, index = df_prices_strategy.index)    
    df_pairs_returns_list.append(df_pairs_returns)

In [ ]:
overall_returns = pd.concat([(df_pairs_returns / df_pairs_returns.shape[1]).sum(axis = 1)  for df_pairs_returns in df_pairs_returns_list])
overall_cum_returns = (1 + overall_returns).cumprod()

In [ ]:
import getFamaFrenchFactors as ff
import statsmodels.api as sm

In [ ]:
returns = overall_returns
returns = returns + 1
returns.index = pd.to_datetime(returns.index)
n_rows = 12 * len(set(returns.index.year))
returns = returns.groupby(by = [returns.index.year, returns.index.month]).prod() - 1

In [ ]:
carhart4_monthly = ff.carhart4Factor(frequency = 'm')
carhart4_monthly["date_ff_factors"] = pd.to_datetime(carhart4_monthly["date_ff_factors"])
carhart4_monthly = carhart4_monthly.iloc[-n_rows:]
carhart4_monthly.set_index("date_ff_factors", inplace = True)

In [ ]:
def reformat_index(s):
    s = s.replace(",", ".")
    if s == ".":
        return math.nan
    return float(s)

In [ ]:
carhart4_monthly["Strategy_return"] = returns.values
carhart4_monthly["Strategy_excess_return"] = carhart4_monthly["Strategy_return"] - carhart4_monthly["RF"]

In [ ]:
X = carhart4_monthly[["Mkt-RF", "SMB", "HML", "MOM"]]
y = carhart4_monthly["Strategy_excess_return"]
X = sm.add_constant(X)
ff_model = sm.OLS(y, X).fit()
print(ff_model.summary())
intercept, b1, b2, b3, b4 = ff_model.params

In [ ]:
strategy_sharpe = math.sqrt(12) * np.average(carhart4_monthly["Strategy_excess_return"]) / np.std(carhart4_monthly["Strategy_excess_return"])

In [ ]:
strategy_sharpe